## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [3]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jan. 19th Jan. 26th


In [4]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [5]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [6]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [7]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [8]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,January-22-2025,Condo/Co-op,400 S Pointe Dr #2304,Miami Beach,FL,33139.0,2800000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,MARMLS,A11730243,N,Y,25.767539,-80.13422


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [9]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
# df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [10]:
df2 = df

In [11]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-01-21    28
2025-01-22    22
2025-01-24    20
2025-01-23    12
2025-01-20     1
2025-01-26     1
2025-01-25     1
Name: count, dtype: int64

In [12]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [13]:
df_top_ten = df2.head(10)

In [14]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
34,PAST SALE,2025-01-24,Condo/Co-op,8701 Collins Ave Ph 1702,Miami Beach,FL,33154.0,29000000.0,6.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8701-Collin...,MARMLS,A11676573,N,Y,25.872197,-80.121210
64,PAST SALE,2025-01-22,Condo/Co-op,100 S Pointe Dr #1409,Miami Beach,FL,33139.0,7700000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11655077,N,Y,25.767180,-80.132822
5,PAST SALE,2025-01-24,Condo/Co-op,495 Brickell Ave Ph -5701,Miami,FL,33131.0,3550000.0,4.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/495-Brickell-A...,MARMLS,A11714596,N,Y,25.768352,-80.188672
85,PAST SALE,2025-01-24,Condo/Co-op,18555 Collins Ave #2803,Sunny Isles Beach,FL,33160.0,3537500.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11440679,N,Y,25.948221,-80.119797
72,PAST SALE,2025-01-22,Condo/Co-op,15701 Collins Ave #4303,Sunny Isles Beach,FL,33160.0,3100000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11624958,N,Y,25.921944,-80.121716
1,PAST SALE,2025-01-22,Condo/Co-op,400 S Pointe Dr #2304,Miami Beach,FL,33139.0,2800000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,MARMLS,A11730243,N,Y,25.767539,-80.134220
2,PAST SALE,2025-01-21,Condo/Co-op,201 Crandon Blvd #166,Key Biscayne,FL,33149.0,2700000.0,4.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,MARMLS,A11685005,N,Y,25.696814,-80.158094
63,PAST SALE,2025-01-24,Condo/Co-op,6000 Island Blvd #2307,Aventura,FL,33160.0,2580000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11660532,N,Y,25.942761,-80.135433
55,PAST SALE,2025-01-22,Condo/Co-op,13611 Deering Bay Dr #1101,Coral Gables,FL,33158.0,2550000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/13611-D...,MARMLS,A11668837,N,Y,25.637553,-80.289763
50,PAST SALE,2025-01-22,Condo/Co-op,2020 N Bayshore Dr #4102,Miami,FL,33137.0,2200000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2020-N-Bayshor...,MARMLS,A11665971,N,Y,25.796683,-80.187669


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [15]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [16]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [17]:
response_df = pd.DataFrame(response_list)

In [18]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [19]:
df_top_ten = merged_df

In [20]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-01-24,Condo/Co-op,8701 Collins Ave Ph 1702,Miami Beach,FL,33154.0,29000000.0,6.0,6.5,...,-80.121210,https://www.redfin.com/FL/Surfside/8701-Collin...,Dina Goldentayer,Douglas Elliman,None,None,Dina Goldentayer,Douglas Elliman,None,None
1,PAST SALE,2025-01-22,Condo/Co-op,100 S Pointe Dr #1409,Miami Beach,FL,33139.0,7700000.0,2.0,2.5,...,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Roland Ortiz,One Sotheby's International Re,Eddy Martinez,One Sotheby's International Re,Raymond Jacomo,Douglas Elliman,None,None
2,PAST SALE,2025-01-24,Condo/Co-op,495 Brickell Ave Ph -5701,Miami,FL,33131.0,3550000.0,4.0,3.0,...,-80.188672,https://www.redfin.com/FL/Miami/495-Brickell-A...,Abe Ash,"Multivest Realty, Inc.",None,None,Milton Andrade,"Compass Florida, LLC.",None,None
3,PAST SALE,2025-01-24,Condo/Co-op,18555 Collins Ave #2803,Sunny Isles Beach,FL,33160.0,3537500.0,3.0,4.5,...,-80.119797,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Melissa Barragan,Dezer Platinum Realty LLC,None,None,Jay Brandt,"Compass Florida, LLC.",None,None
4,PAST SALE,2025-01-22,Condo/Co-op,15701 Collins Ave #4303,Sunny Isles Beach,FL,33160.0,3100000.0,2.0,2.5,...,-80.121716,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,Maria Mariana,Fortune International Realty,Jessie Gesen,Fortune International Realty,Miltiadis Kastanis,Douglas Elliman,None,None
5,PAST SALE,2025-01-22,Condo/Co-op,400 S Pointe Dr #2304,Miami Beach,FL,33139.0,2800000.0,3.0,3.0,...,-80.134220,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,Nina Winiger,London Foster Realty,None,None,JTHS Accounting,MIAMI ASSOCIATION OF REALTORS®,None,None
6,PAST SALE,2025-01-21,Condo/Co-op,201 Crandon Blvd #166,Key Biscayne,FL,33149.0,2700000.0,4.0,4.0,...,-80.158094,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,German Kaupert,London Real Estate Co.,None,None,Patricia Peraita,Coldwell Banker Realty,None,None
7,PAST SALE,2025-01-24,Condo/Co-op,6000 Island Blvd #2307,Aventura,FL,33160.0,2580000.0,3.0,3.5,...,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Roman Garcia,Brickss Real Estate Company,None,None,Karen Matluck,"Compass Florida, LLC",None,None
8,PAST SALE,2025-01-22,Condo/Co-op,13611 Deering Bay Dr #1101,Coral Gables,FL,33158.0,2550000.0,3.0,3.0,...,-80.289763,https://www.redfin.com/FL/Coral-Gables/13611-D...,Geri Brodie,Onyx Realty,None,None,Berenice Elguezabal,Coldwell Banker Realty,None,None
9,PAST SALE,2025-01-22,Condo/Co-op,2020 N Bayshore Dr #4102,Miami,FL,33137.0,2200000.0,3.0,3.5,...,-80.187669,https://www.redfin.com/FL/Miami/2020-N-Bayshor...,Joanne Walker,Coldwell Banker Realty,None,None,Christopher Zdancewicz,One Sotheby's International Re,None,None


## Current Week's Values

In [21]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

85


In [22]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,105,012


In [23]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$598


In [24]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$93,926,000


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [25]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
106
Input Previous Week Condo Average Sales Price:
899049
Input Previous Week Condo Average PSF:
602
Input Previous Week Condo Sales total (ex: 198_000_000)
61_900_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [26]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [27]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [28]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [29]:
pd.set_option('display.max_columns',None)

In [30]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-24,Condo/Co-op,8701 Collins Ave Ph 1702,Miami Beach,FL,33154.0,29000000.0,6.0,6.5,8701 COLLINS AVE CONDO,6380.0,NaN,2020.0,NaN,4545.0,21095.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8701-Collin...,MARMLS,A11676573,N,Y,25.872197,-80.12121,https://www.redfin.com/FL/Surfside/8701-Collin...,Dina Goldentayer,Douglas Elliman,None,None,Dina Goldentayer,Douglas Elliman,None,None,1,orange


In [31]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [32]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-24,Condo/Co-op,8701 Collins Ave Ph 1702,Miami Beach,FL,33154.0,29000000.0,6.0,6.5,8701 COLLINS AVE CONDO,6380.0,NaN,2020.0,NaN,4545.0,21095.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8701-Collin...,MARMLS,A11676573,N,Y,25.872197,-80.121210,https://www.redfin.com/FL/Surfside/8701-Collin...,Dina Goldentayer,Douglas Elliman,None,None,Dina Goldentayer,Douglas Elliman,None,None,1,orange
1,PAST SALE,2025-01-22,Condo/Co-op,100 S Pointe Dr #1409,Miami Beach,FL,33139.0,7700000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1595.0,NaN,2002.0,NaN,4828.0,4119.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11655077,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Roland Ortiz,One Sotheby's International Re,Eddy Martinez,One Sotheby's International Re,Raymond Jacomo,Douglas Elliman,None,None,2,blue
2,PAST SALE,2025-01-24,Condo/Co-op,495 Brickell Ave Ph -5701,Miami,FL,33131.0,3550000.0,4.0,3.0,ICON BRICKELL NO TWO,2632.0,NaN,2009.0,NaN,1349.0,2755.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/495-Brickell-A...,MARMLS,A11714596,N,Y,25.768352,-80.188672,https://www.redfin.com/FL/Miami/495-Brickell-A...,Abe Ash,"Multivest Realty, Inc.",None,None,Milton Andrade,"Compass Florida, LLC.",None,None,3,blue
3,PAST SALE,2025-01-24,Condo/Co-op,18555 Collins Ave #2803,Sunny Isles Beach,FL,33160.0,3537500.0,3.0,4.5,18555 COLLINS AVENUE COND,3171.0,NaN,2016.0,NaN,1116.0,5033.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11440679,N,Y,25.948221,-80.119797,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Melissa Barragan,Dezer Platinum Realty LLC,None,None,Jay Brandt,"Compass Florida, LLC.",None,None,4,blue
4,PAST SALE,2025-01-22,Condo/Co-op,15701 Collins Ave #4303,Sunny Isles Beach,FL,33160.0,3100000.0,2.0,2.5,15701 COLLINS CONDO,1447.0,NaN,2020.0,NaN,2142.0,2189.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11624958,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,Maria Mariana,Fortune International Realty,Jessie Gesen,Fortune International Realty,Miltiadis Kastanis,Douglas Elliman,None,None,5,blue
5,PAST SALE,2025-01-22,Condo/Co-op,400 S Pointe Dr #2304,Miami Beach,FL,33139.0,2800000.0,3.0,3.0,SOUTH POINTE TOWERS CONDO,1802.0,NaN,1987.0,NaN,1554.0,2648.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,MARMLS,A11730243,N,Y,25.767539,-80.134220,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,Nina Winiger,London Foster Realty,None,None,JTHS Accounting,MIAMI ASSOCIATION OF REALTORS®,None,None,6,blue
6,PAST SALE,2025-01-21,Condo/Co-op,201 Crandon Blvd #166,Key Biscayne,FL,33149.0,2700000.0,4.0,4.0,KEY COLONY NO 1,3075.0,NaN,1978.0,NaN,878.0,6530.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,MARMLS,A11685005,N,Y,25.696814,-80.158094,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,German Kaupert,London Real Estate Co.,None,None,Patricia Peraita,Coldwell Banker Realty,None,None,7,blue
7,PAST SALE,2025-01-24,Condo/Co-op,6000 Island Blvd #2307,Aventura,FL,33160.0,2580000.0,3.0,3.5,BELLAMARE @ WILLIAMS ISLA,2840.0,NaN,2005.0,NaN,908.0,2500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11660532,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Roman Garcia,Brickss Real Estate Company,None,None,Karen Matluck,"Compass Florida, LLC",None,None,8,blue
8,PAST SALE,2025-01-22,Condo/Co-op,13611 Deering Bay Dr #1101,Coral Gables,FL,33158.0,2550000.0,3.0,3.0,SIENA AT DEERING BAY COND,2907.0,NaN,20

In [33]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [34]:
m.save('index.html')

## Data snagger

In [35]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [36]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-24,Condo/Co-op,8701 Collins Ave Ph 1702,Miami Beach,FL,33154.0,29000000.0,6.0,6.5,8701 COLLINS AVE CONDO,6380.0,NaN,2020.0,NaN,4545.0,21095.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8701-Collin...,MARMLS,A11676573,N,Y,25.872197,-80.12121,https://www.redfin.com/FL/Surfside/8701-Collin...,Dina Goldentayer,Douglas Elliman,None,None,Dina Goldentayer,Douglas Elliman,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [37]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_01272024


# CREATE TEMPLATE 

In [38]:
muni_set = set(df_top_ten['CITY'])

In [39]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [40]:
df_top_ten.reset_index(inplace=True,drop=True)

In [41]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [42]:
top_sale

'8701 COLLINS AVE CONDO at 8701 Collins Ave Ph 1702 in Miami Beach'

In [43]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [44]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [45]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-24,Condo/Co-op,8701 Collins Ave Ph 1702,Miami Beach,FL,33154.0,29000000.0,6.0,6.5,8701 COLLINS AVE CONDO,6380.0,NaN,2020.0,NaN,4545.0,21095.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8701-Collin...,MARMLS,A11676573,N,Y,25.872197,-80.121210,https://www.redfin.com/FL/Surfside/8701-Collin...,Dina Goldentayer,Douglas Elliman,None,None,Dina Goldentayer,Douglas Elliman,None,None,1,orange,8701 COLLINS AVE CONDO at 8701 Collins Ave Ph ...
1,PAST SALE,2025-01-22,Condo/Co-op,100 S Pointe Dr #1409,Miami Beach,FL,33139.0,7700000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1595.0,NaN,2002.0,NaN,4828.0,4119.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11655077,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Roland Ortiz,One Sotheby's International Re,Eddy Martinez,One Sotheby's International Re,Raymond Jacomo,Douglas Elliman,None,None,2,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #1...
2,PAST SALE,2025-01-24,Condo/Co-op,495 Brickell Ave Ph -5701,Miami,FL,33131.0,3550000.0,4.0,3.0,ICON BRICKELL NO TWO,2632.0,NaN,2009.0,NaN,1349.0,2755.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/495-Brickell-A...,MARMLS,A11714596,N,Y,25.768352,-80.188672,https://www.redfin.com/FL/Miami/495-Brickell-A...,Abe Ash,"Multivest Realty, Inc.",None,None,Milton Andrade,"Compass Florida, LLC.",None,None,3,blue,ICON BRICKELL NO TWO at 495 Brickell Ave Ph -5...
3,PAST SALE,2025-01-24,Condo/Co-op,18555 Collins Ave #2803,Sunny Isles Beach,FL,33160.0,3537500.0,3.0,4.5,18555 COLLINS AVENUE COND,3171.0,NaN,2016.0,NaN,1116.0,5033.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11440679,N,Y,25.948221,-80.119797,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Melissa Barragan,Dezer Platinum Realty LLC,None,None,Jay Brandt,"Compass Florida, LLC.",None,None,4,blue,18555 COLLINS AVENUE COND at 18555 Collins Ave...
4,PAST SALE,2025-01-22,Condo/Co-op,15701 Collins Ave #4303,Sunny Isles Beach,FL,33160.0,3100000.0,2.0,2.5,15701 COLLINS CONDO,1447.0,NaN,2020.0,NaN,2142.0,2189.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11624958,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,Maria Mariana,Fortune International Realty,Jessie Gesen,Fortune International Realty,Miltiadis Kastanis,Douglas Elliman,None,None,5,blue,15701 COLLINS CONDO at 15701 Collins Ave #4303...
5,PAST SALE,2025-01-22,Condo/Co-op,400 S Pointe Dr #2304,Miami Beach,FL,33139.0,2800000.0,3.0,3.0,SOUTH POINTE TOWERS CONDO,1802.0,NaN,1987.0,NaN,1554.0,2648.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,MARMLS,A11730243,N,Y,25.767539,-80.134220,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,Nina Winiger,London Foster Realty,None,None,JTHS Accounting,MIAMI ASSOCIATION OF REALTORS®,None,None,6,blue,SOUTH POINTE TOWERS CONDO at 400 S Pointe Dr #...
6,PAST SALE,2025-01-21,Condo/Co-op,201 Crandon Blvd #166,Key Biscayne,FL,33149.0,2700000.0,4.0,4.0,KEY COLONY NO 1,3075.0,NaN,1978.0,NaN,878.0,6530.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,MARMLS,A11685005,N,Y,25.696814,-80.158094,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,German Kaupert,London Real Estate Co.,None,None,Patricia Peraita,Coldwell Banker Realty,None,None,7,blue,KEY COLONY NO 1 at 201 Crandon Blvd #166 in Ke...
7,PAST SALE,2025-01-24,Condo/Co-op,6000 Island Blvd #2307,Aventura,FL,33160.0,2580000.0,3.0,3.5,BELLAMARE @ WILLIAMS ISLA,2840.0,NaN,2005.0,NaN,908.0,2500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aven

In [46]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 8701 COLLINS AVE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,200,000 to $29,000,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Sunny Isles Beach, Key Biscayne, Aventura, Miami, Coral Gables, Miami Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 85 condo sales totaling $93,926,000 million from Jan. 19th to Jan. 26th. The previous week, brokers closed 106 condo sales totaling $61,900,000.

Last week’s units sold for an average of $1,105,012, higher than the $899,049 average price f

In [47]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [48]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [49]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [50]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-24,Condo/Co-op,8701 Collins Ave Ph 1702,Miami Beach,FL,33154.0,29000000.0,6.0,6.5,8701 COLLINS AVE CONDO,6380.0,NaN,2020.0,NaN,4545.0,21095.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8701-Collin...,MARMLS,A11676573,N,Y,25.872197,-80.121210,https://www.redfin.com/FL/Surfside/8701-Collin...,Dina Goldentayer,Douglas Elliman,None,None,Dina Goldentayer,Douglas Elliman,None,None,1,orange,8701 COLLINS AVE CONDO at 8701 Collins Ave Ph ...
1,PAST SALE,2025-01-22,Condo/Co-op,100 S Pointe Dr #1409,Miami Beach,FL,33139.0,7700000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1595.0,NaN,2002.0,NaN,4828.0,4119.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11655077,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Roland Ortiz,One Sotheby's International Re,Eddy Martinez,One Sotheby's International Re,Raymond Jacomo,Douglas Elliman,None,None,2,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #1...
2,PAST SALE,2025-01-24,Condo/Co-op,495 Brickell Ave Ph -5701,Miami,FL,33131.0,3550000.0,4.0,3.0,ICON BRICKELL NO TWO,2632.0,NaN,2009.0,NaN,1349.0,2755.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/495-Brickell-A...,MARMLS,A11714596,N,Y,25.768352,-80.188672,https://www.redfin.com/FL/Miami/495-Brickell-A...,Abe Ash,"Multivest Realty, Inc.",None,None,Milton Andrade,"Compass Florida, LLC.",None,None,3,blue,ICON BRICKELL NO TWO at 495 Brickell Ave Ph -5...
3,PAST SALE,2025-01-24,Condo/Co-op,18555 Collins Ave #2803,Sunny Isles Beach,FL,33160.0,3537500.0,3.0,4.5,18555 COLLINS AVENUE COND,3171.0,NaN,2016.0,NaN,1116.0,5033.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11440679,N,Y,25.948221,-80.119797,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Melissa Barragan,Dezer Platinum Realty LLC,None,None,Jay Brandt,"Compass Florida, LLC.",None,None,4,blue,18555 COLLINS AVENUE COND at 18555 Collins Ave...
4,PAST SALE,2025-01-22,Condo/Co-op,15701 Collins Ave #4303,Sunny Isles Beach,FL,33160.0,3100000.0,2.0,2.5,15701 COLLINS CONDO,1447.0,NaN,2020.0,NaN,2142.0,2189.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11624958,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,Maria Mariana,Fortune International Realty,Jessie Gesen,Fortune International Realty,Miltiadis Kastanis,Douglas Elliman,None,None,5,blue,15701 COLLINS CONDO at 15701 Collins Ave #4303...
5,PAST SALE,2025-01-22,Condo/Co-op,400 S Pointe Dr #2304,Miami Beach,FL,33139.0,2800000.0,3.0,3.0,SOUTH POINTE TOWERS CONDO,1802.0,NaN,1987.0,NaN,1554.0,2648.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,MARMLS,A11730243,N,Y,25.767539,-80.134220,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,Nina Winiger,London Foster Realty,None,None,JTHS Accounting,MIAMI ASSOCIATION OF REALTORS®,None,None,6,blue,SOUTH POINTE TOWERS CONDO at 400 S Pointe Dr #...
6,PAST SALE,2025-01-21,Condo/Co-op,201 Crandon Blvd #166,Key Biscayne,FL,33149.0,2700000.0,4.0,4.0,KEY COLONY NO 1,3075.0,NaN,1978.0,NaN,878.0,6530.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,MARMLS,A11685005,N,Y,25.696814,-80.158094,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,German Kaupert,London Real Estate Co.,None,None,Patricia Peraita,Coldwell Banker Realty,None,None,7,blue,KEY COLONY NO 1 at 201 Crandon Blvd #166 in Ke...
7,PAST SALE,2025-01-24,Condo/Co-op,6000 Island Blvd #2307,Aventura,FL,33160.0,2580000.0,3.0,3.5,BELLAMARE @ WILLIAMS ISLA,2840.0,NaN,2005.0,NaN,908.0,2500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aven

In [51]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: 8701 COLLINS AVE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,200,000 to $29,000,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Sunny Isles Beach, Key Biscayne, Aventura, Miami, Coral Gables, Miami Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 85 condo sales totaling $93,926,000 million from Jan. 19th to Jan. 26th. The previous week, brokers closed 106 condo sales totaling $61,900,000.

Last week’s units sold for an average of $1,105,012, higher than the $899,049 average price f

In [59]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Surfside/8701-Collins-Ave-33154/unit-1702/home/146032762


In [60]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/100-S-Pointe-Dr-33139/unit-1409/home/43288629


In [61]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/2020-N-Bayshore-Dr-33137/unit-4102/home/45429184


In [62]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Coral-Gables/13611-Deering-Bay-Dr-33158/unit-1101/home/43264562


In [63]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/100-S-Pointe-Dr-33139/unit-1409/home/43288629


In [64]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Coral-Gables/13611-Deering-Bay-Dr-33158/unit-1101/home/43264562


In [65]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-24,Condo/Co-op,8701 Collins Ave Ph 1702,Miami Beach,FL,33154.0,29000000.0,6.0,6.5,8701 COLLINS AVE CONDO,6380.0,NaN,2020.0,NaN,4545.0,21095.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8701-Collin...,MARMLS,A11676573,N,Y,25.872197,-80.121210,https://www.redfin.com/FL/Surfside/8701-Collin...,Dina Goldentayer,Douglas Elliman,None,None,Dina Goldentayer,Douglas Elliman,None,None,1,orange,8701 COLLINS AVE CONDO at 8701 Collins Ave Ph ...
1,PAST SALE,2025-01-22,Condo/Co-op,100 S Pointe Dr #1409,Miami Beach,FL,33139.0,7700000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1595.0,NaN,2002.0,NaN,4828.0,4119.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11655077,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Roland Ortiz,One Sotheby's International Re,Eddy Martinez,One Sotheby's International Re,Raymond Jacomo,Douglas Elliman,None,None,2,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #1...
2,PAST SALE,2025-01-24,Condo/Co-op,495 Brickell Ave Ph -5701,Miami,FL,33131.0,3550000.0,4.0,3.0,ICON BRICKELL NO TWO,2632.0,NaN,2009.0,NaN,1349.0,2755.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/495-Brickell-A...,MARMLS,A11714596,N,Y,25.768352,-80.188672,https://www.redfin.com/FL/Miami/495-Brickell-A...,Abe Ash,"Multivest Realty, Inc.",None,None,Milton Andrade,"Compass Florida, LLC.",None,None,3,blue,ICON BRICKELL NO TWO at 495 Brickell Ave Ph -5...
3,PAST SALE,2025-01-24,Condo/Co-op,18555 Collins Ave #2803,Sunny Isles Beach,FL,33160.0,3537500.0,3.0,4.5,18555 COLLINS AVENUE COND,3171.0,NaN,2016.0,NaN,1116.0,5033.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11440679,N,Y,25.948221,-80.119797,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Melissa Barragan,Dezer Platinum Realty LLC,None,None,Jay Brandt,"Compass Florida, LLC.",None,None,4,blue,18555 COLLINS AVENUE COND at 18555 Collins Ave...
4,PAST SALE,2025-01-22,Condo/Co-op,15701 Collins Ave #4303,Sunny Isles Beach,FL,33160.0,3100000.0,2.0,2.5,15701 COLLINS CONDO,1447.0,NaN,2020.0,NaN,2142.0,2189.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11624958,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,Maria Mariana,Fortune International Realty,Jessie Gesen,Fortune International Realty,Miltiadis Kastanis,Douglas Elliman,None,None,5,blue,15701 COLLINS CONDO at 15701 Collins Ave #4303...
5,PAST SALE,2025-01-22,Condo/Co-op,400 S Pointe Dr #2304,Miami Beach,FL,33139.0,2800000.0,3.0,3.0,SOUTH POINTE TOWERS CONDO,1802.0,NaN,1987.0,NaN,1554.0,2648.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,MARMLS,A11730243,N,Y,25.767539,-80.134220,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,Nina Winiger,London Foster Realty,None,None,JTHS Accounting,MIAMI ASSOCIATION OF REALTORS®,None,None,6,blue,SOUTH POINTE TOWERS CONDO at 400 S Pointe Dr #...
6,PAST SALE,2025-01-21,Condo/Co-op,201 Crandon Blvd #166,Key Biscayne,FL,33149.0,2700000.0,4.0,4.0,KEY COLONY NO 1,3075.0,NaN,1978.0,NaN,878.0,6530.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,MARMLS,A11685005,N,Y,25.696814,-80.158094,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,German Kaupert,London Real Estate Co.,None,None,Patricia Peraita,Coldwell Banker Realty,None,None,7,blue,KEY COLONY NO 1 at 201 Crandon Blvd #166 in Ke...
7,PAST SALE,2025-01-24,Condo/Co-op,6000 Island Blvd #2307,Aventura,FL,33160.0,2580000.0,3.0,3.5,BELLAMARE @ WILLIAMS ISLA,2840.0,NaN,2005.0,NaN,908.0,2500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aven

## Time on Market Calculator

In [67]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 9, 10) ## List (Earlier) date
date2 = datetime(2025, 1, 22) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

134


## Clean CSV for Datawrapper Chart

In [68]:
chart_df = df_top_ten

In [69]:
chart_df = chart_df.fillna(" ")

In [70]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [71]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [72]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [73]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [74]:
chart_df['$/SQUARE FEET'].astype(float)

0    4545.0
1    4828.0
2    1349.0
3    1116.0
4    2142.0
5    1554.0
6     878.0
7     908.0
8     877.0
9    1000.0
Name: $/SQUARE FEET, dtype: float64

In [75]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [76]:
csv_date_string = today.strftime("%m_%d_%Y")

In [77]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [78]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-01-24,8701 Collins Ave Ph 1702 in Miami Beach,"$29,000,000",6,6.5,"6,380",2020,"$4,545",MARMLS,A11676573,Dina Goldentayer Douglas Elliman,Dina Goldentayer Douglas Elliman
1,2025-01-22,100 S Pointe Dr #1409 in Miami Beach,"$7,700,000",2,2.5,"1,595",2002,"$4,828",MARMLS,A11655077,Roland Ortiz One Sotheby's International Re Ed...,Raymond Jacomo Douglas Elliman
2,2025-01-24,495 Brickell Ave Ph -5701 in Miami,"$3,550,000",4,3.0,"2,632",2009,"$1,349",MARMLS,A11714596,"Abe Ash Multivest Realty, Inc.","Milton Andrade Compass Florida, LLC."
3,2025-01-24,18555 Collins Ave #2803 in Sunny Isles Beach,"$3,537,500",3,4.5,"3,171",2016,"$1,116",MARMLS,A11440679,Melissa Barragan Dezer Platinum Realty LLC,"Jay Brandt Compass Florida, LLC."
4,2025-01-22,15701 Collins Ave #4303 in Sunny Isles Beach,"$3,100,000",2,2.5,"1,447",2020,"$2,142",MARMLS,A11624958,Maria Mariana Fortune International Realty Jes...,Miltiadis Kastanis Douglas Elliman
5,2025-01-22,400 S Pointe Dr #2304 in Miami Beach,"$2,800,000",3,3.0,"1,802",1987,"$1,554",MARMLS,A11730243,Nina Winiger London Foster Realty,JTHS Accounting MIAMI ASSOCIATION OF REALTORS®...
6,2025-01-21,201 Crandon Blvd #166 in Key Biscayne,"$2,700,000",4,4.0,"3,075",1978,$878,MARMLS,A11685005,German Kaupert London Real Estate Co.,Patricia Peraita Coldwell Banker Realty
7,2025-01-24,6000 Island Blvd #2307 in Aventura,"$2,580,000",3,3.5,"2,840",2005,$908,MARMLS,A11660532,Roman Garcia Brickss Real Estate Company,"Karen Matluck Compass Florida, LLC"
8,2025-01-22,13611 Deering Bay Dr #1101 in Coral Gables,"$2,550,000",3,3.0,"2,907",2001,$877,MARMLS,A11668837,Geri Brodie Onyx Realty,Berenice Elguezabal Coldwell Banker Realty
9,2025-01-22,2020 N Bayshore Dr #4102 in Miami,"$2,200,000",3,3.5,"2,201",2010,"$1,000",MARMLS,A11665971,Joanne Walker Coldwell Banker Realty,Christopher Zdancewicz One Sotheby's Internati...
